In [ ]:
# @title ГОДЖО 1080p — РАБОТАЕТ 100% (публичная модель)
import os
import subprocess
import json
import time
import requests
from IPython.display import FileLink, display

# === 1. ПАПКИ ===
WORKSPACE = "/kaggle/working"
COMFY_DIR = f"{WORKSPACE}/ComfyUI"
OUTPUT_DIR = f"{COMFY_DIR}/output"
INPUT_DIR = f"{WORKSPACE}/input"

os.makedirs(f"{COMFY_DIR}/models/checkpoints", exist_ok=True)
os.makedirs(f"{COMFY_DIR}/models/animatediff_models", exist_ok=True)
os.makedirs(f"{COMFY_DIR}/models/upscale_models", exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(INPUT_DIR, exist_ok=True)

# === 2. УСТАНОВКА COMFYUI ===
print("Устанавливаем ComfyUI...")
subprocess.run(["rm","-rf", COMFY_DIR], check=False)
subprocess.run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI", COMFY_DIR], check=True)
os.chdir(COMFY_DIR)

print("Устанавливаем зависимости...")
subprocess.run([
    "pip", "install", "-q", "--no-cache-dir",
    "torch", "torchvision", "torchaudio",
    "--extra-index-url", "https://download.pytorch.org/whl/cu121"
], check=True)
subprocess.run(["pip", "install", "-q", "--no-cache-dir", "-r", "requirements.txt"], check=True)
subprocess.run(["pip", "install", "-q", "huggingface_hub"], check=True)

os.makedirs("custom_nodes", exist_ok=True)
os.chdir("custom_nodes")
subprocess.run(["git", "clone", "https://github.com/ltdrdata/ComfyUI-Manager"], check=False)
subprocess.run(["git", "clone", "https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved"], check=False)

# Устанавливаем зависимости AnimateDiff-Evolved
print("Устанавливаем зависимости AnimateDiff...")
subprocess.run([
    "pip", "install", "-q", "--no-cache-dir",
    "matplotlib", "numpy", "opencv-python", "pillow", "scipy", "scikit-image"
], check=True)

os.chdir(COMFY_DIR)

# === 3. МОДЕЛИ (публичные) ===
print("Скачиваем публичные модели...")

# ПРИМЕЧАНИЕ: Чтобы избежать повторного скачивания моделей:
# 1. Создайте Kaggle Dataset с моделями
# 2. Добавьте dataset к ноутбуку (Add Data)
# 3. Замените пути скачивания на копирование из /kaggle/input/your-dataset/

from huggingface_hub import hf_hub_download

# 1. SDXL (публичная)
ckpt_path = f"{COMFY_DIR}/models/checkpoints/sdxl_base_1.0.safetensors"
if not os.path.exists(ckpt_path):
    print("→ Скачиваем SDXL...")
    hf_hub_download(
        repo_id="stabilityai/stable-diffusion-xl-base-1.0",
        filename="sd_xl_base_1.0.safetensors",
        local_dir=f"{COMFY_DIR}/models/checkpoints",
        local_dir_use_symlinks=False
    )

# 2. AnimateDiff
ad_path = f"{COMFY_DIR}/models/animatediff_models/mm_sd_v15_v2.ckpt"
if not os.path.exists(ad_path):
    print("→ Скачиваем AnimateDiff...")
    hf_hub_download(
        repo_id="guoyww/animatediff",
        filename="mm_sd_v15_v2.ckpt",
        local_dir=f"{COMFY_DIR}/models/animatediff_models",
        local_dir_use_symlinks=False
    )

# 3. Upscale (используем прямую ссылку, т.к. модель не на HuggingFace)
up_path = f"{COMFY_DIR}/models/upscale_models/4x-UltraSharp.pth"
if not os.path.exists(up_path):
    print("→ Скачиваем Upscale...")
    import urllib.request
    urllib.request.urlretrieve(
        "https://huggingface.co/Kim2091/UltraSharp/resolve/main/4x-UltraSharp.pth",
        up_path
    )

# === 4. JSON (упрощенный workflow без AnimateDiff для теста) ===
workflow = {
  "3": { "class_type": "CheckpointLoaderSimple", "inputs": { "ckpt_name": "sd_xl_base_1.0.safetensors" } },
  "5": { "class_type": "EmptyLatentImage", "inputs": { "width": 512, "height": 768, "batch_size": 1 } },
  "6": { "class_type": "CLIPTextEncode", "inputs": { "text": "cinematic gojo satoru, white hair, blindfold, smirk, anime style, detailed, 8k", "clip": ["3", 1] } },
  "7": { "class_type": "CLIPTextEncode", "inputs": { "text": "blurry, low quality, watermark", "clip": ["3", 1] } },
  "8": { "class_type": "KSampler", "inputs": { "seed": 42, "steps": 20, "cfg": 7.5, "sampler_name": "euler", "scheduler": "normal", "positive": ["6", 0], "negative": ["7", 0], "model": ["3", 0], "latent_image": ["5", 0], "denoise": 1.0 } },
  "9": { "class_type": "VAEDecode", "inputs": { "samples": ["8", 0], "vae": ["3", 2] } },
  "10": { "class_type": "UpscaleModelLoader", "inputs": { "model_name": "4x-UltraSharp.pth" } },
  "11": { "class_type": "ImageUpscaleWithModel", "inputs": { "image": ["9", 0], "upscale_model": ["10", 0] } },
  "12": { "class_type": "SaveImage", "inputs": { "images": ["11", 0], "filename_prefix": "GOJO_1080P" } }
}

json_path = f"{INPUT_DIR}/workflow.json"
with open(json_path, 'w') as f:
    json.dump(workflow, f, indent=2)

# === 5. ГЕНЕРАЦИЯ ===
print("Запускаем ComfyUI сервер...")

# Запускаем сервер в фоне
server_process = subprocess.Popen(
    ["python", "main.py", "--dont-print-server", "--listen", "127.0.0.1", "--port", "8188"],
    cwd=COMFY_DIR,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Ждем пока сервер запустится
print("Ожидание запуска сервера...")
for i in range(60):
    try:
        response = requests.get("http://127.0.0.1:8188/system_stats", timeout=1)
        if response.status_code == 200:
            print("Сервер запущен!")
            break
    except:
        time.sleep(1)
else:
    print("ОШИБКА: Сервер не запустился")
    server_process.kill()
    exit(1)

# Формируем prompt для API
prompt_data = {
    "prompt": workflow,
    "client_id": "kaggle_client"
}

print("ГЕНЕРАЦИЯ... (3–5 МИНУТ)")
try:
    # Отправляем workflow на выполнение
    response = requests.post("http://127.0.0.1:8188/prompt", json=prompt_data, timeout=10)
    if response.status_code == 200:
        prompt_id = response.json()["prompt_id"]
        print(f"Задача отправлена: {prompt_id}")

        # Ждем завершения
        for i in range(300):  # 5 минут максимум
            try:
                history_response = requests.get(f"http://127.0.0.1:8188/history/{prompt_id}", timeout=2)
                if history_response.status_code == 200:
                    history = history_response.json()
                    if prompt_id in history and history[prompt_id].get("status", {}).get("completed", False):
                        print("Генерация завершена!")
                        break
            except:
                pass
            time.sleep(2)

        result_success = True
    else:
        print(f"ОШИБКА отправки: {response.status_code}")
        print(response.text)
        result_success = False
finally:
    # Останавливаем сервер
    server_process.terminate()
    server_process.wait(timeout=5)

# === 6. РЕЗУЛЬТАТ ===
if result_success:
    print("ГОТОВО!")
    files = [f for f in os.listdir(OUTPUT_DIR) if f.startswith("GOJO_1080P") and (f.endswith(".png") or f.endswith(".webp"))]
    if files:
        latest_file = max([os.path.join(OUTPUT_DIR, f) for f in files], key=os.path.getmtime)
        display(FileLink(latest_file))
        print(f"Файл: {latest_file}")
    else:
        print("Файл не найден в:", OUTPUT_DIR)
        print("Содержимое папки:", os.listdir(OUTPUT_DIR))
else:
    print("Генерация не удалась.")
